# Exploring conformational space of selected macrocycles - "M1"; <br /> Part 1: Generation and selection of conformer candidates (MM methods)

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import glob
import py3Dmol

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

In [ ]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def write_to_dict(prefix, suppl):
    moldict = {}
    for i, mol in enumerate(suppl):
        name = prefix + str(i)
        moldict[name] = mol
    return moldict


def align_structures_to_crystal(moldict):
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))


def align_structures_to_lowest_energy(moldict, energy_dict):
    """
    align all structures in "moldict" to the one of the lowest energy
    """
    energy_sorted = sorted(energy_dict.items(), key=lambda x: x[1])
    first = energy_sorted[0][0]
    core_first = moldict[first].GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    
    for key, mol in moldict.items():
        core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,moldict[first],atomMap=list(zip(core_mol,core_first)))


def prepare_view(moldict):
    p = py3Dmol.view(width=400,height=400)
    for key, mol in moldict.items():
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.15'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p        


def make_similarity_matrix(moldict):
    
    similarity_matrix = {}

    for k1, m1 in moldict.items():
        for k2, m2 in moldict.items():
            if (k1, k2) in similarity_matrix.keys() or (k2, k1) in similarity_matrix.keys():
                pass
            else:
                if k1 != k2:
                    rms = AllChem.GetBestRMS(Chem.RemoveHs(m1),Chem.RemoveHs(m2))
                    similarity_matrix[(k1, k2)] = rms 
                    
    return similarity_matrix


def find_duplicates_in_sorted_similarity_matrix(similarity_matrix_sorted, energy):
    
    similarity_thresh = 0.5 # Angstrom
    energy_thresh     = 5   # kcal/mol
    
    to_be_deleted     = []
    
    for pair in similarity_matrix_sorted:
        if pair[1] < similarity_thresh:
            conf1 = pair[0][0]
            conf2 = pair[0][1]
            if abs(energy[conf1] - energy[conf2]) < energy_thresh:
                #print("conf1, conf2, E(conf1), E(conf2) = ", conf1, conf2, energy_m1_b[conf1], energy_m1_b[conf2])
                if energy[conf1] < energy[conf2]:
                    to_be_deleted.append(conf2)
                else:
                    to_be_deleted.append(conf1)
        else:
            # similarity_matrix_b_sorted is sorted, so here we would already start looping over 
            # pairs for which rmsd is > threshold 
            # and we do not need to do this, so break
            break

    return to_be_deleted


def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

## Crystal structure of "M1" macrocycle

In [ ]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.removeAllModels()
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

In [ ]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

m1 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.sdf')
m1_crystal = templ_m1[0]

## Conformers generated with the Balloon software:

Conformers were generated in the Balloon software with the following starting structures:

* the crystal geometry (3D), results with prefix: "m1_b_sdf"; the crystal is of the "ss-ss" type;

* the SMILES signature of M1 ("0D") and allowing to "rebuild the geometry" (option "--rebuildGeometry"), results with prefix: "m1_b_smi"

* 3D structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

    where "ss\_sa" means "(ring1: syn-syn)\_(ring2: syn-anti)" configuration of the amide groups, paired by the neighbouring aromatic ring, etc.
    
The geometries of generated conformers were then pre-optimized with the MM methods (using MMFF94-like force field).

In all cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

In [ ]:
inps_m1_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsdf/*.sdf')
inps_m1_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsmiles/*.sdf')
inps_m1_b_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_sa/*.sdf')
inps_m1_b_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_ss_aa/*.sdf')
inps_m1_b_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_sa/*.sdf')
inps_m1_b_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_as/*.sdf')
inps_m1_b_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_sa_aa/*.sdf')
inps_m1_b_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_m1_aa_aa/*.sdf')

In [ ]:
e_m1_b_sdf = grep_energies_from_sdf_outputs(inps_m1_b_sdf)
e_m1_b_smi = grep_energies_from_sdf_outputs(inps_m1_b_smi)
e_m1_b_ss_sa = grep_energies_from_sdf_outputs(inps_m1_b_ss_sa)
e_m1_b_ss_aa = grep_energies_from_sdf_outputs(inps_m1_b_ss_aa)
e_m1_b_sa_sa = grep_energies_from_sdf_outputs(inps_m1_b_sa_sa)
e_m1_b_sa_as = grep_energies_from_sdf_outputs(inps_m1_b_sa_as)
e_m1_b_sa_aa = grep_energies_from_sdf_outputs(inps_m1_b_sa_aa)
e_m1_b_aa_aa = grep_energies_from_sdf_outputs(inps_m1_b_aa_aa)

In [ ]:
# write conformers to dictionaries
   
suppl_m1_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_sdfout.sdf')
suppl_m1_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_smilesout.sdf')
suppl_m1_b_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_sa_sdfout.sdf')
suppl_m1_b_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_ss_aa_sdfout.sdf')
suppl_m1_b_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_sa_sdfout.sdf')
suppl_m1_b_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_as_sdfout.sdf')
suppl_m1_b_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_sa_aa_sdfout.sdf')
suppl_m1_b_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_aa_aa_sdfout.sdf')

allmol_m1_b_sdf   = write_to_dict("m1_b_sdf_", suppl_m1_b_sdf)
allmol_m1_b_smi   = write_to_dict("m1_b_smi_", suppl_m1_b_smi)
allmol_m1_b_ss_sa = write_to_dict("m1_b_ss_sa_", suppl_m1_b_ss_sa)
allmol_m1_b_ss_aa = write_to_dict("m1_b_ss_aa_", suppl_m1_b_ss_aa)
allmol_m1_b_sa_sa = write_to_dict("m1_b_sa_sa_", suppl_m1_b_sa_sa)
allmol_m1_b_sa_as = write_to_dict("m1_b_sa_as_", suppl_m1_b_sa_as)
allmol_m1_b_sa_aa = write_to_dict("m1_b_sa_aa_", suppl_m1_b_sa_aa)
allmol_m1_b_aa_aa = write_to_dict("m1_b_aa_aa_", suppl_m1_b_aa_aa)

The generated conformer structures are presented in a separate notebook: [link](http://nbviewer.jupyter.org/github/gosiao/icho-notebooks/blob/master/conformers_m1_suppl1.ipynb).

### pre-screening

All generated structures are pre-optimized with MM methods (MMFF94-like force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account non-hydrogen atoms only);
* compare the energies of conformers from pairs which are found to be similar (RMSD lower than the threshold);
* if these energies are too similar (the difference lower than the threshold), we will remove the conformer which has the higher energy value;

In [ ]:
allmol_m1_b = {}
allmol_m1_b.update(allmol_m1_b_sdf)
allmol_m1_b.update(allmol_m1_b_smi)
allmol_m1_b.update(allmol_m1_b_ss_sa)
allmol_m1_b.update(allmol_m1_b_ss_aa)
allmol_m1_b.update(allmol_m1_b_sa_sa)
allmol_m1_b.update(allmol_m1_b_sa_as)
allmol_m1_b.update(allmol_m1_b_sa_aa)
allmol_m1_b.update(allmol_m1_b_aa_aa)

print("The total number of generated conformers = ", len(allmol_m1_b))
             
energy_m1_b = {}
energy_m1_b.update(e_m1_b_sdf)
energy_m1_b.update(e_m1_b_smi)
energy_m1_b.update(e_m1_b_ss_sa)
energy_m1_b.update(e_m1_b_ss_aa)
energy_m1_b.update(e_m1_b_sa_sa)
energy_m1_b.update(e_m1_b_sa_as)
energy_m1_b.update(e_m1_b_sa_aa)
energy_m1_b.update(e_m1_b_aa_aa)

In [ ]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_b = make_similarity_matrix(allmol_m1_b)
similarity_matrix_b_sorted = sorted(similarity_matrix_b.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy
to_be_deleted = find_duplicates_in_sorted_similarity_matrix(similarity_matrix_b_sorted, energy_m1_b)

for mol in to_be_deleted:
    #print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_b.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_b[k]
    allmol_m1_b.pop(mol, None)
    energy_m1_b.pop(mol, None)
    

In [ ]:
print("We have removed potential conformer duplicates.")
print("The final number of remaining conformers = ", len(allmol_m1_b))
print("Below we present all the remaining conformers aligned (to one aromatic ring).")

In [ ]:
align_structures_to_lowest_energy(allmol_m1_b, energy_m1_b)
p_b = prepare_view(allmol_m1_b)
p_b.show()

In [ ]:
print("Sorted energy of all selected conformers and the energy differences with respect to the lowest:")
energy_sorted = sorted(energy_m1_b.items(), key=lambda x: x[1])
energy_diff = []
e_min = energy_sorted[0][1]
for e in energy_sorted:
    e_diff = e[1] - e_min
    energy_diff.append([e[0], e[1], e_diff])

e_df = pd.DataFrame(energy_diff, columns=["conformer", "E", "E - Emin"])
e_df

## Conformers generated with the RDKit software:

Conformers were generated in the RDKit software with the following starting structures:

* the crystal geometry (3D), results with prefix: "m1_b_sdf"; the crystal is of the "ss-ss" type;

* the SMILES signature of M1 ("0D") and allowing to "rebuild the geometry" (option "--rebuildGeometry"), results with prefix: "m1_b_smi"

* 3D structures generated in Avogadro (from the crystal geometry and pre-optimized) of the:
    * "ss_sa" type
    * "ss_aa" type
    * "sa_sa" type
    * "sa_as" type
    * "sa_aa" type
    * "aa_aa" type    

    where "ss\_sa" means "(ring1: syn-syn)\_(ring2: syn-anti)" configuration of the amide groups, paired by the neighbouring aromatic ring, etc.


The geometries of generated conformers were then pre-optimized with the MM methods (using UFF force field).

In all cases the RDKit software was asked to generate 100 conformers using the distnce geometry algorithm with default settings (only "pruneRmsThresh" set to 1.0 in "AllChem.EmbedMultipleConfs" and "maxIters" set to 500 or 700 (when needed) in "AllChem.UFFOptimizeMolecule").

In [ ]:
inps_m1_rdkit_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_smiles/*.sdf')
inps_m1_rdkit_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_sdf/*.sdf')
inps_m1_rdkit_ss_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_sa/*.sdf')
inps_m1_rdkit_ss_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_ss_aa/*.sdf')
inps_m1_rdkit_sa_sa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_sa/*.sdf')
inps_m1_rdkit_sa_as = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_as/*.sdf')
inps_m1_rdkit_sa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_sa_aa/*.sdf')
inps_m1_rdkit_aa_aa = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_from_m1_aa_aa/*.sdf')

In [ ]:
e_m1_rdkit_smi = grep_energies_from_sdf_outputs(inps_m1_rdkit_smi)
e_m1_rdkit_sdf = grep_energies_from_sdf_outputs(inps_m1_rdkit_sdf)
e_m1_rdkit_ss_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_sa)
e_m1_rdkit_ss_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_ss_aa)
e_m1_rdkit_sa_sa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_sa)
e_m1_rdkit_sa_as = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_as)
e_m1_rdkit_sa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_sa_aa)
e_m1_rdkit_aa_aa = grep_energies_from_sdf_outputs(inps_m1_rdkit_aa_aa)

In [ ]:
# write conformers to dictionaries
suppl_m1_rdkit_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_smiles.sdf')
suppl_m1_rdkit_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_sdf.sdf')
suppl_m1_rdkit_ss_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_sa.sdf')
suppl_m1_rdkit_ss_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_ss_aa.sdf')
suppl_m1_rdkit_sa_sa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_sa.sdf')
suppl_m1_rdkit_sa_as  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_as.sdf')
suppl_m1_rdkit_sa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_sa_aa.sdf')
suppl_m1_rdkit_aa_aa  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_m1_aa_aa.sdf')

allmol_m1_rdkit_smi = write_to_dict("m1_rdkit_smi_", suppl_m1_rdkit_smi)
allmol_m1_rdkit_sdf = write_to_dict("m1_rdkit_sdf_", suppl_m1_rdkit_sdf)
allmol_m1_rdkit_ss_sa = write_to_dict("m1_rdkit_ss_sa_", suppl_m1_rdkit_ss_sa)
allmol_m1_rdkit_ss_aa = write_to_dict("m1_rdkit_ss_aa_", suppl_m1_rdkit_ss_aa)
allmol_m1_rdkit_sa_sa = write_to_dict("m1_rdkit_sa_sa_", suppl_m1_rdkit_sa_sa)
allmol_m1_rdkit_sa_as = write_to_dict("m1_rdkit_sa_as_", suppl_m1_rdkit_sa_as)
allmol_m1_rdkit_sa_aa = write_to_dict("m1_rdkit_sa_aa_", suppl_m1_rdkit_sa_aa)
allmol_m1_rdkit_aa_aa = write_to_dict("m1_rdkit_aa_aa_", suppl_m1_rdkit_aa_aa)

The generated conformer structures are presented in a separate notebook: [link](http://nbviewer.jupyter.org/github/gosiao/icho-notebooks/blob/master/conformers_m1_suppl1.ipynb).

### pre-screening

All the generated structures are pre-optimized with MM methods (UFF force field). To remove potential duplicates, we will:

* calculate the root-mean-square-distance (RMSD) between the pairs of conformers (taking into account non-hydrogen atoms only);
* compare the energies of conformers from pairs which are found to be similar (RMSD lower than the threshold);
* if these energies are too similar (the difference lower than the threshold), we will remove the conformer which has the higher energy value;

In [ ]:
allmol_m1_rdkit = {}
allmol_m1_rdkit.update(allmol_m1_rdkit_sdf)
allmol_m1_rdkit.update(allmol_m1_rdkit_smi)
allmol_m1_rdkit.update(allmol_m1_rdkit_ss_sa)
allmol_m1_rdkit.update(allmol_m1_rdkit_ss_aa)
allmol_m1_rdkit.update(allmol_m1_rdkit_sa_sa)
allmol_m1_rdkit.update(allmol_m1_rdkit_sa_as)
allmol_m1_rdkit.update(allmol_m1_rdkit_sa_aa)
allmol_m1_rdkit.update(allmol_m1_rdkit_aa_aa)

print("The total number of generated conformers = ", len(allmol_m1_rdkit))
             
energy_m1_rdkit = {}
energy_m1_rdkit.update(e_m1_rdkit_sdf)
energy_m1_rdkit.update(e_m1_rdkit_smi)
energy_m1_rdkit.update(e_m1_rdkit_ss_sa)
energy_m1_rdkit.update(e_m1_rdkit_ss_aa)
energy_m1_rdkit.update(e_m1_rdkit_sa_sa)
energy_m1_rdkit.update(e_m1_rdkit_sa_as)
energy_m1_rdkit.update(e_m1_rdkit_sa_aa)
energy_m1_rdkit.update(e_m1_rdkit_aa_aa)

In [ ]:
# 1. calculate the similarity matrix between all pairs of conformers and sort its elements from the lowest
# (the most similar structures) to the largest values (the most different structures)
similarity_matrix_rdkit = make_similarity_matrix(allmol_m1_rdkit)
similarity_matrix_rdkit_sorted = sorted(similarity_matrix_rdkit.items(), key=lambda x: x[1])

# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than threshold ("similarity_thresh"), 
# compare energies; then if the energies are similar (controlled by the "energy_thresh"), 
#then remove the one with higher energy
to_be_deleted = find_duplicates_in_sorted_similarity_matrix(similarity_matrix_rdkit_sorted, energy_m1_rdkit)

for mol in to_be_deleted:
    #print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix_rdkit.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix_rdkit[k]
    allmol_m1_rdkit.pop(mol, None)
    energy_m1_rdkit.pop(mol, None) 

In [ ]:
print("We have removed potential conformer duplicates.")
print("The final number of remaining conformers = ", len(allmol_m1_rdkit))
print("Below we present all the remaining conformers aligned (to one aromatic ring).")

In [ ]:
align_structures_to_lowest_energy(allmol_m1_rdkit, energy_m1_rdkit)
p_rdkit = prepare_view(allmol_m1_rdkit)
p_rdkit.show()

## Summary

Now let's generate a list of all conformers (from all programs used).
We will not further pre-screen the structures, as different programs use different force fields and the comparison of structures based on RMSD without the comparison of energies might lead to the removal of promising candidates.

In [ ]:
allmol_m1 = {}
allmol_m1.update(allmol_m1_b)
allmol_m1.update(allmol_m1_rdkit)

energy_m1 = {}
energy_m1.update(energy_m1_b)
energy_m1.update(energy_m1_rdkit)

Finally we can align all conformers which will further be used as starting points in DFT geometry optimizations; the total number of conformers is:

In [ ]:
print(len(allmol_m1))

In [ ]:
align_structures_to_lowest_energy(allmol_m1, energy_m1)
p_all = prepare_view(allmol_m1)
p_all.show()

Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". It will be used to generate Gaussian inputs:

In [ ]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m1.items():
        f.write(key+"\n")

energy_sorted = sorted(energy_m1.items(), key=lambda x: x[1])
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m1/detailed_list_selected_conformers_from_ballon_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1]))        